In [1]:
try:
    from rich import load_ipython_extension
    %load_ext rich
except ImportError:
    try:
        from rich import pretty
        pretty.install()
    except ImportError:
        pass

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score

Using the cleaned dataset created in assignment 1

In [3]:
data = pd.read_csv("../data/manipulated_data.csv")
data

,sex,age,income_bracket,year,total_sum_sek_millions,number_of_persons,region_name,level_of_education
0,Man,16-19,TOT,2018,547.9,38091,Stockholm,primary/lower secondary
1,Man,16-19,0,2018,0.0,13357,Stockholm,primary/lower secondary
2,Man,16-19,1-19,2018,137.9,17417,Stockholm,primary/lower secondary
3,Man,16-19,20-39,2018,106.1,3763,Stockholm,primary/lower secondary
4,Man,16-19,40-59,2018,77.2,1574,Stockholm,primary/lower secondary
...,...,...,...,...,...,...,...,...
409255,Woman,65-69,80-99,2012,0.9,10,Norrbotten,unknown
409256,Woman,70-75,TOT,2012,2.4,36,Norrbotten,unknown
409257,Woman,70-75,0,2012,0.0,11,Norrbotten,unknown
409258,Woman,70-75,1-19,2012,0.1,6,Norrbotten,unknown


I will try to classify which age the test instances has based on their features.

In [4]:
data["age"].unique().tolist()

['16-19', '20-29', '30-39', '40-49', '50-59', '60-64', '65-69', '70-75']

In [5]:
labels = data[["age"]]
labels

,age
0,16-19
1,16-19
2,16-19
3,16-19
4,16-19
...,...
409255,65-69
409256,70-75
409257,70-75
409258,70-75


I will use all features in the first experiment.

In [6]:
features = data.drop(columns="age")
features

,sex,income_bracket,year,total_sum_sek_millions,number_of_persons,region_name,level_of_education
0,Man,TOT,2018,547.9,38091,Stockholm,primary/lower secondary
1,Man,0,2018,0.0,13357,Stockholm,primary/lower secondary
2,Man,1-19,2018,137.9,17417,Stockholm,primary/lower secondary
3,Man,20-39,2018,106.1,3763,Stockholm,primary/lower secondary
4,Man,40-59,2018,77.2,1574,Stockholm,primary/lower secondary
...,...,...,...,...,...,...,...
409255,Woman,80-99,2012,0.9,10,Norrbotten,unknown
409256,Woman,TOT,2012,2.4,36,Norrbotten,unknown
409257,Woman,0,2012,0.0,11,Norrbotten,unknown
409258,Woman,1-19,2012,0.1,6,Norrbotten,unknown


In [7]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, train_size=0.7, random_state=42, shuffle=True)

X_train

,sex,income_bracket,year,total_sum_sek_millions,number_of_persons,region_name,level_of_education
252845,Man,60-79,2015,16.0,231,Kronoberg,post-secondary
285224,Woman,1000+,2021,286.7,219,Skåne,upper secondary
326532,Woman,160-179,2010,28.7,170,Norrbotten,post-secondary
74032,Man,1-19,2023,0.2,21,Örebro,unknown
171175,Man,80-99,2014,10.3,107,Södermanland,unknown
...,...,...,...,...,...,...,...
259178,Man,240-259,2015,34.1,136,Halland,post-secondary
365838,Man,340-359,2022,1.1,3,Blekinge,unknown
131932,Man,800-999,2020,3.4,4,Dalarna,primary/lower secondary
146867,Man,360-379,2017,166.5,450,Kronoberg,upper secondary


In [8]:
y_train

,age
252845,20-29
285224,50-59
326532,20-29
74032,20-29
171175,20-29
...,...
259178,30-39
365838,20-29
131932,30-39
146867,20-29


In [ ]:
# Parameter Suggested Range:

learning_rate = [0.01, 0.05, 0.1, 0.2]
max_iter = [100, 300, 500, 1000]
max_leaf_nodes = [10, 20, 31, 50, 100] # default = 31
max_depth = [3, 5, 8, None]
min_samples_leaf = [1, 5, 10, 20, 50]
l2_regularization = [0, 1e-3, 1e-2, 1e-1, 1]
max_features = [0.3, 0.5, 0.7, 1.0]
early_stopping = [True, False]
validation_fraction = [0.1, 0.2]
n_iter_no_change = [5, 10, 20]

Catagorical features

In [13]:
cat_features = features.drop(columns=["year", "total_sum_sek_millions", "number_of_persons"]).columns.to_list()

cat_features

['sex', 'income_bracket', 'region_name', 'level_of_education']

In [14]:
HGB_clf = HistGradientBoostingClassifier(
    loss="log_loss", learning_rate=0.1, max_iter=100, categorical_features=cat_features, l2_regularization=0, validation_fraction=0.1, n_iter_no_change=10, verbose=1, random_state=42
).fit(X=X_train, y=y_train)

/home/joel/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Binning 0.014 GB of training data: 0.065 s
Binning 0.002 GB of validation data: 0.003 s
Fitting gradient boosted rounds:
Fit 800 trees in 8.873 s, (24800 total leaves)
Time spent computing histograms: 1.871s
Time spent finding best splits:  0.308s
Time spent applying splits:      1.262s
Time spent predicting:           0.301s


In [15]:
HGB_clf.score(X=X_test, y=y_test)

0.44525892260828487